# Generating target points

In [1]:
import os
import sys
import time

import laspy
from laspy.file import File

import numpy as np

In [2]:
# global variables

workingdirectory="D:/Koma/escience/test_escience_2018Nov/exampledata/"
filename="tile_00022"

In [3]:
start1 = time.time()
# read las file

in_pc = File(workingdirectory+filename+".las", mode='r')
in_pc_nparray = np.vstack([in_pc.x, in_pc.y, in_pc.z]).transpose()

print(("Number of points are: %s") % (in_pc_nparray.shape[0]))

# generate grid in 2D z==0 (for 2.5D analysis within cylinder)

min_x=np.min(in_pc_nparray[:,0])
max_x=np.max(in_pc_nparray[:,0])
step_x=1

min_y=np.min(in_pc_nparray[:,1])
max_y=np.max(in_pc_nparray[:,1])
step_y=1

bound_x = np.arange(min_x, max_x, step_x)
bound_y = np.arange(min_y, max_y, step_y)

target_x, target_y = np.meshgrid(bound_x, bound_y, indexing='ij')

# export as XYZ pcloud

x=np.ravel(target_x)
y=np.ravel(target_y)
z=np.ones(len(x))

false_intensity=np.zeros(len(x))

out_LAS = File(workingdirectory+filename+"_target.las", mode = "w", header = in_pc.header)
out_LAS.x = x
out_LAS.y = y
out_LAS.z = z
out_LAS.intensity = false_intensity
out_LAS.close()

end1 = time.time()
difftime1=end1 - start1
print(("create target point: %f sec") % (difftime1))

Number of points are: 3708822
create target point: 0.483163 sec


# Development branch test - raster

In [4]:
sys.path.insert(0, "D:/GitHub/eEcoLiDAR/develop-branch/eEcoLiDAR/")

from laserchicken import read_las
from laserchicken.keys import point
from laserchicken.volume_specification import Cell,InfiniteCylinder
from laserchicken.compute_neighbors import compute_neighborhoods
from laserchicken.feature_extractor import compute_features
from laserchicken.write_ply import write

In [12]:
start1 = time.time()
print("------ Import is started ------")

pc = read_las.read(workingdirectory+filename+".las")
target = read_las.read(workingdirectory+filename+"_target.las")
radius=1

print(("Number of points: %s ") % (pc[point]['x']['data'].shape[0]))
print(("Number of points in target: %s ") % (target[point]['x']['data'].shape[0]))

print("------ Computing neighborhood is started ------")

#compute_neighborhoods is now a generator. To get the result of a generator the user
#needs to call next(compute_neighborhoods). The following shows how to get the results.
#
#indices_cyl=compute_neighborhoods(pc, target, Cell(np.float(args.radius)))
#
neighbors=compute_neighborhoods(pc, target, Cell(np.float(radius)))
iteration=0
target_idx_base=0
for x in neighbors:
    end = time.time()
    difftime=end - start
    print(("build kd-tree: %f sec") % (difftime))
    print("Computed neighborhoods list length at iteration %d is: %d" % (iteration,len(x)))

    start1 = time.time()
    print("------ Feature calculation is started ------")
    compute_features(pc, x, target_idx_base, target, ['max_z','min_z','mean_z','median_z','std_z','var_z','coeff_var_z','skew_z','kurto_z','eigenv_1','eigenv_2','eigenv_3'], Cell(np.float(radius)))
    target_idx_base+=len(x)
    end1 = time.time()
    difftime1=end1 - start1
    print(("feature calc: %f sec") % (difftime1))
    iteration+=1


write(target,workingdirectory+filename+"_cell_1m.ply")

end1 = time.time()
difftime1=end1 - start1
print(("feature calc: %f sec") % (difftime1))

------ Import is started ------
Number of points: 14649421 
Number of points in target: 1010025 
------ Computing neighborhood is started ------
Cylinder size in Bytes: 6346174239.884063
Memory size in Bytes: 17056247808
Start tree creation
Done with env tree creation
Done with target tree creation
build kd-tree: 709.784516 sec
Computed neighborhoods list length at iteration 0 is: 1010025
------ Feature calculation is started ------
Feature "eigenv_3"
Number of targets: 1010025, number of features: 3
 took 136.02 seconds
Feature "kurto_z"
Number of targets: 1010025, number of features: 1
 took 152.07 seconds
Feature "skew_z"
Number of targets: 1010025, number of features: 1
 took 210.50 seconds
Feature "coeff_var_z"
Number of targets: 1010025, number of features: 3


D:/GitHub/eEcoLiDAR/develop-branch/eEcoLiDAR\laserchicken\feature_extractor\mean_std_coeff_z_feature_extractor.py:23: RuntimeWarning: divide by zero encountered in double_scalars
  coeff_var_z = std_z / mean_z
D:/GitHub/eEcoLiDAR/develop-branch/eEcoLiDAR\laserchicken\feature_extractor\mean_std_coeff_z_feature_extractor.py:23: RuntimeWarning: invalid value encountered in double_scalars
  coeff_var_z = std_z / mean_z


 took 40.32 seconds
Feature "var_z"
Number of targets: 1010025, number of features: 1
 took 28.20 seconds
Feature "median_z"
Number of targets: 1010025, number of features: 1
 took 48.68 seconds
Feature "min_z"
Number of targets: 1010025, number of features: 3
 took 13.28 seconds
The following unrequested features were calculated as a side effect, but will not be returned: ['range_z']
feature calc: 629.066668 sec
feature calc: 674.716810 sec


# Development branch - point cloud based

In [9]:
start = time.time()
print("------ Import is started ------")

pc = read_las.read(workingdirectory+filename+".las")
radius=1

print(("Number of points: %s ") % (pc[point]['x']['data'].shape[0]))
#print(("Number of points in target: %s ") % (target[point]['x']['data'].shape[0]))

print("------ Computing neighborhood is started ------")

#compute_neighborhoods is now a generator. To get the result of a generator the user
#needs to call next(compute_neighborhoods). The following shows how to get the results.
#
#indices_cyl=compute_neighborhoods(pc, target, Cell(np.float(args.radius)))
#
neighbors=compute_neighborhoods(pc, pc, InfiniteCylinder(np.float(radius)))
iteration=0
target_idx_base=0
for x in neighbors:
    end = time.time()
    difftime=end - start
    print(("build kd-tree: %f sec") % (difftime))
    print("Computed neighborhoods list length at iteration %d is: %d" % (iteration,len(x)))

    start1 = time.time()
    print("------ Feature calculation is started ------")
    compute_features(pc, x, target_idx_base, pc, ['max_z','min_z','mean_z','median_z','std_z','var_z','coeff_var_z','skew_z','kurto_z','range_z','eigenv_1','eigenv_2','eigenv_3'], InfiniteCylinder(np.float(radius)))
    target_idx_base+=len(x)
    end1 = time.time()
    difftime1=end1 - start1
    print(("feature calc: %f sec") % (difftime1))
    iteration+=1


write(target,workingdirectory+filename+"_cylinder_1m.ply")

end1 = time.time()
difftime1=end1 - start1
print(("feature calc: %f sec") % (difftime1))

------ Import is started ------
Number of points: 3708822 
------ Computing neighborhood is started ------
Cylinder size in Bytes: 46606431794.68881
Memory size in Bytes: 17056247808
Number of points: 678646
build kd-tree: 19.782990 sec
Computed neighborhoods list length at iteration 0 is: 678646
------ Feature calculation is started ------
Feature "eigenv_3"
Number of targets: 678646, number of features: 3
 took 117.67 seconds
Feature "range_z"
Number of targets: 678646, number of features: 3
 took 12.33 seconds
Feature "kurto_z"
Number of targets: 678646, number of features: 1
 took 111.60 seconds
Feature "skew_z"
Number of targets: 678646, number of features: 1
 took 149.87 seconds
Feature "coeff_var_z"
Number of targets: 678646, number of features: 3
 took 31.93 seconds
Feature "var_z"
Number of targets: 678646, number of features: 1
 took 22.25 seconds
Feature "median_z"
Number of targets: 678646, number of features: 1
 took 39.78 seconds
The following unrequested features were ca

NameError: name 'target' is not defined